## Вводная информация 
#### Контекст эксперимента:

Компания "N" провела эксперимент.  
При открытии сайта компании одной части пользователей показывалось стандартное всплывающее окно с анкетой, другой части пользователей было показано новая версия всплывающего окна.
В обеих случаях пользователи могли ответить "yes" или "no" на задаваемый в анкете вопрос, или закрыть всплывающее окно, не ответив.  

Повлияла ли новая версия анкеты на конверсию?

#### Комментарий:
Данные взяты из открытого [источника](https://www.kaggle.com/datasets/osuolaleemmanuel/ad-ab-testing).


## Исследование данных собранных в ходе эксперимента 

#### Описание столбцов:
- __auction_id__ - уникальный идентификатор пользователя, присвоенный при демонстрации всплывающего окна с анкетой;
- __experiment__ - группа пользователей ("control" - контрольная, "exposed" - экспериментальная);
- __date__ - дата;
- __hour__ - час;
- __device_make__ - устройство на котором был открыт сайт;
- __platform_os__ - идентификатор ОС устройства;
- __browser__ - браузер, используемый пользователем;
- __yes__ - пользователь ответил утвердительно на вопрос анкеты (0 - нет, 1 - да);
- __no__ - пользователь ответил отрицательно на вопрос анкеты (0 - нет, 1 - да).

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [5]:
# загрузка датафрейма 
df = pd.read_csv(r"data\ab_data_1.csv")
df

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


In [148]:
# время проведения эксперимента 
df['date'].nunique()

# количество демонстраций в разные дни недели 
df['date'] = pd.to_datetime(df['date'])
df['date'].dt.day_name().value_counts()

## эксперимент длился 8 дней
## А-Б тест длиться больше недели, это позволяет учесть различие в поведении пользователей в разные дни недели

Friday       2908
Thursday     1208
Wednesday    1198
Saturday      903
Sunday        890
Monday        490
Tuesday       480
Name: date, dtype: int64

In [4]:
# какими браузерами пользовались участники эксперимента 

df['browser'].value_counts()

## браузеры, используемые не часто, объединяем в одну группу "OTHER"
valid_br = df['browser'].value_counts().index[:5]
df['browser'] = df['browser'].apply(lambda a: a if a in valid_br else 'OTHER')

df['browser'].value_counts(normalize=True)

## более половины пользователей в ходе эксперимента воспользовались "Chrome Mobile"

Chrome Mobile            0.563823
Chrome Mobile WebView    0.184351
Samsung Internet         0.102018
Facebook                 0.094590
Mobile Safari            0.041723
OTHER                    0.013495
Name: browser, dtype: float64

In [6]:
# добавление столбеца "pass", определяющего совершил пользователь целевое действие или нет 
df['pass'] = df.apply(lambda row: 0 if (row['yes']==0)&(row['no']==0) else 1, axis=1)

# разделение датафрейма на экспериментальную и контрольную группу 
df_control = df[df['experiment']=='control']
df_exposed = df[df['experiment']=='exposed']

In [8]:
# экспериментальная группа
print('Экспериментальная группа:', df_exposed['pass'].value_counts(normalize=True), sep='\n', end='\n\n')

# контрольная группа
print('Контрольная группа:', df_control['pass'].value_counts(normalize=True), sep='\n')

## в экспериментальной группе конверсия составляет 16%, в контрольной группе - 14%
## т.е. размер эффекта составил 2 

Экспериментальная группа:
0    0.835996
1    0.164004
Name: pass, dtype: float64

Контрольная группа:
0    0.856055
1    0.143945
Name: pass, dtype: float64


## Проверка гипотезы о влиянии новой версии анкеты на конверсию

H<sub>0</sub>: Процент людей, прошедших опрос, в обоих группах не отличается  
H<sub>А</sub>: Процент людей, прошедших опрос, в экспериментальной группе выше по сравнению с контрольной

In [43]:
## для проверки гипотезы воспользуемся Z-критерием для разности долей в независимых выборках
from statsmodels.stats.proportion import proportions_ztest

# размер контрольной выборки 
control_len = len(df_control)
# кол-во людей, прошедших опрос, в контрольной выборке 
control_pass = len(df_control[df_control['pass']==1])

# размер экспериментальной выборки 
exposed_len = len(df_exposed)
# кол-во людей, прошедших опрос, в экспериментальной выборке 
exposed_pass = len(df_exposed[df_exposed['pass']==1])

# уровень значимости
alpha = 0.05 

In [10]:
# проверка гипотезы о равенстве долей 
_, p_val =  proportions_ztest(count=(exposed_pass, control_pass), nobs=(exposed_len, control_len))
print(f"P-значение: {p_val:.3}") 
## p-value меньше определенного ранее уровня значимости. нулевая гипотеза гипотеза отвергается
## доли людей, ответивших на вопрос, в двух группах не равны 

P-значение: 0.0125


In [11]:
# проверка гипотезы о равенстве долей, с указанием альтернативной гипотезы: доля прошедших опрос в экспериментальной группе выше чем в контрольной
_, p_val =  proportions_ztest(count=(exposed_pass, control_pass), nobs=(exposed_len, control_len), alternative='larger')
print(f"P-значение: {p_val:.3}") 
## p-value меньше определенного ранее уровня значимости, нулевая гипотеза отвергается в пользу альтернативной

P-значение: 0.00625


In [42]:
# расчет мощности проведенного эксперимента 

# функция "get_power" упрощает работу с функцией "zt_ind_solve_power" из библиотеки "statsmodels"
def get_power(l_1, l_2, g_1, g_2, alpha, alternative='two-sided'):
    
    from statsmodels.stats.power import zt_ind_solve_power
    
    # доля пользователей, ответивших на вопрос из анкеты
    p_1 = g_1 / l_1
    p_2 = g_2 / l_2 
    
    # дисперсии
    var_1 = p_1 * (1-p_1)
    var_2 = p_2 * (1-p_2)
    
    # стандартизированный размер эффекта
    st_ef = (p_2 - p_1) / ((l_1*np.sqrt(var_1) + l_2*np.sqrt(var_2)) / (l_1 + l_2))

    return zt_ind_solve_power(effect_size=st_ef, # стандартизованный размер эффекта
                       nobs1=l_1 + l_2,          # суммарное число наблюдений
                       alpha=alpha,              # уровень значимости
                       power=None,               # мощность (None, т.к. ее необходимо найти)
                       ratio=l_1 / l_2,          # отношение размеров выборок
                       alternative=alternative)  # альтернатива
        
get_power(len(df_control), 
          len(df_exposed), 
          df_control['pass'].value_counts()[1], 
          df_exposed['pass'].value_counts()[1],
          alpha=0.05, alternative='larger')    
## при уровне значимость 0.05, мощность проведенного эксперимента равна 0.97 

0.9715829560005882

Проверка результата результат расчета мощности.  
На основании полученной мощности, рассчитаем заранее известный нам размере выборки. Расчет размера выборки, необходимого для получения заданных ошибок первого и второго рода, в данном случае, будет выполняться по формуле:  

$$n = \left(\frac{{Z_{1-\alpha} \cdot \sqrt{{p_{\text{0}} \cdot (1 - p_{\text{0}})}} + Z_{1-\beta} \cdot \sqrt{{p_{\text{a}} \cdot (1 - p_{\text{a}})}}}}{{p_{\text{a}} - p_{\text{0}}}}\right)^2$$

где:  
__n__ - число наблюдений;  
__α__ - ошибка первого рода;  
__β__ - ошибка второго рода;  
__z<sub>1-α</sub>,  z<sub>1-β</sub>__ - квантили нормального распределения;  
__p<sub>0</sub>__,  __p<sub>a</sub>__ - отношения ответов на вопрос к показам окна в контрольной группе и в экспериментальной группе;  
__(p<sub>0</sub> - p<sub>a</sub>)__ - размер эффекта, который хотим задетектить.

In [15]:
# функция расчета необходимого объема выборки для достижения требуемых значений ошибки первого и второго рода 

def get_size(p0, pa, alpha, beta):
    za = stats.norm.ppf(1 - alpha)
    zb = stats.norm.ppf(1 - beta)    
    n = za * np.sqrt(p0*(1 - p0)) + zb * np.sqrt(pa*(1-pa))
    n = n / (pa - p0) 
    return int(np.ceil(n*n))

alpha = 0.05
beta = 0.03 # 1 - 0.97 (полученное значение мощности)

get_size(df_control['pass'].value_counts(normalize=True)[1],
         df_exposed['pass'].value_counts(normalize=True)[1], 
         alpha, beta)
## результат расчета, требуемого размера одной группы, равен 4033 (при условии что две группы будут одинаковые). фактические размеры групп равны 4071, 4006
## по результату проверки можно сделать вывод, о том что мощность эксперимента рассчитана верно 

4033

## Влияние используемого браузера на конверсию при проведении эксперимента 
Проверим гипотезу о равенстве конверсий в контрольной и экспериментальной группе, с учетом браузера, в котором пользователи открывали сайт.

In [27]:
# сводная таблица кол-ва показов анкеты в разных браузерах 

browser_tab = df_exposed.groupby(['browser']).agg({'pass':'count'}).merge(
    df_control.groupby(['browser']).agg({'pass':'count'}), 
    left_index=True, right_index=True,  suffixes=('_exposed', '_control'))

browser_tab

,pass_exposed,pass_control
browser,,
Chrome Mobile,2144,2410
Chrome Mobile WebView,1197,292
Facebook,203,561
Mobile Safari,91,246
OTHER,39,70
Samsung Internet,332,492


In [28]:
# сводная таблица кол-ва пропусков и ответов на вопрос анкеты
browser_tab_dit = df_exposed.groupby(['browser']).agg({'pass':'value_counts'}).merge(
    df_control.groupby(['browser']).agg({'pass':'value_counts'}), 
    left_index=True, right_index=True,  suffixes=('_exposed', '_control'))

browser_tab_dit

pass_exposed  pass_control
browser               pass                            
Chrome Mobile         0             1773          2086
                      1              371           324
Chrome Mobile WebView 0             1017           245
                      1              180            47
Facebook              0              159           449
                      1               44           112
Mobile Safari         0               87           236
                      1                4            10
OTHER                 0               38            65
                      1                1             5
Samsung Internet      0              275           404
                      1               57            88

In [ ]:
# проверка гипотезы о равенстве конверсий в контрольной и экспериментальной группе для каждого браузера 

In [93]:
_, p_val =  proportions_ztest(count=(browser_tab_dit.loc['Chrome Mobile', 'pass_exposed'][1],browser_tab_dit.loc['Chrome Mobile', 'pass_control'][1]), 
                              nobs=(browser_tab.loc['Chrome Mobile', 'pass_exposed'], browser_tab.loc['Chrome Mobile', 'pass_control']), alternative='larger')
print(f"P-значение: {p_val:.3}") 
print('Мощьность:', get_power(browser_tab.loc['Chrome Mobile', 'pass_control'],
          browser_tab.loc['Chrome Mobile', 'pass_exposed'],
          browser_tab_dit.loc['Chrome Mobile', 'pass_control'][1],
          browser_tab_dit.loc['Chrome Mobile', 'pass_exposed'][1],
          alpha=0.05, alternative='larger'))

P-значение: 0.00015
мощьность: 0.9998633200807792


In [106]:
_, p_val =  proportions_ztest(count=(browser_tab_dit.loc['Chrome Mobile WebView', 'pass_exposed'][1],browser_tab_dit.loc['Chrome Mobile WebView', 'pass_control'][1]), 
                              nobs=(browser_tab.loc['Chrome Mobile WebView', 'pass_exposed'], browser_tab.loc['Chrome Mobile WebView', 'pass_control']), alternative='smaller')
print(f"P-значение: {p_val:.3}") 
print('Мощьность:', get_power(browser_tab.loc['Chrome Mobile WebView', 'pass_control'],
          browser_tab.loc['Chrome Mobile WebView', 'pass_exposed'],
          browser_tab_dit.loc['Chrome Mobile WebView', 'pass_control'][1],
          browser_tab_dit.loc['Chrome Mobile WebView', 'pass_exposed'][1],
          alpha=0.05, alternative='smaller'))

P-значение: 0.326
мощьность: 0.12679098371579473


In [110]:
_, p_val =  proportions_ztest(count=(browser_tab_dit.loc['Facebook', 'pass_exposed'][1],browser_tab_dit.loc['Facebook', 'pass_control'][1]), 
                              nobs=(browser_tab.loc['Facebook', 'pass_exposed'], browser_tab.loc['Facebook', 'pass_control']), alternative='larger')
print(f"P-значение: {p_val:.3}") 
print('Мощьность:', get_power(browser_tab.loc['Facebook', 'pass_control'],
          browser_tab.loc['Facebook', 'pass_exposed'],
          browser_tab_dit.loc['Facebook', 'pass_control'][1],
          browser_tab_dit.loc['Facebook', 'pass_exposed'][1],
          alpha=0.05, alternative='larger'))

P-значение: 0.302
мощьность: 0.26123956220629674


In [135]:
_, p_val =  proportions_ztest(count=(browser_tab_dit.loc['Mobile Safari', 'pass_exposed'][1],browser_tab_dit.loc['Mobile Safari', 'pass_control'][1]), 
                              nobs=(browser_tab.loc['Mobile Safari', 'pass_exposed'], browser_tab.loc['Mobile Safari', 'pass_control']), alternative='larger')
print(f"P-значение: {p_val:.3}") 
print('Мощьность:', get_power(browser_tab.loc['Mobile Safari', 'pass_control'],
          browser_tab.loc['Mobile Safari', 'pass_exposed'],
          browser_tab_dit.loc['Mobile Safari', 'pass_control'][1],
          browser_tab_dit.loc['Mobile Safari', 'pass_exposed'][1],
          alpha=0.05, alternative='larger')) 

P-значение: 0.446
мощьность: 0.08302908202596099


In [142]:
_, p_val =  proportions_ztest(count=(browser_tab_dit.loc['Samsung Internet', 'pass_exposed'][1],browser_tab_dit.loc['Samsung Internet', 'pass_control'][1]), 
                              nobs=(browser_tab.loc['Samsung Internet', 'pass_exposed'], browser_tab.loc['Samsung Internet', 'pass_control']), alternative='smaller')
print(f"P-значение: {p_val:.3}") 
print('Мощьность:', get_power(browser_tab.loc['Samsung Internet', 'pass_control'],
          browser_tab.loc['Samsung Internet', 'pass_exposed'],
          browser_tab_dit.loc['Samsung Internet', 'pass_control'][1],
          browser_tab_dit.loc['Samsung Internet', 'pass_exposed'][1],
          alpha=0.05, alternative='smaller'))

P-значение: 0.395
мощьность: 0.10993357712883428


In [146]:
_, p_val =  proportions_ztest(count=(browser_tab_dit.loc['OTHER', 'pass_exposed'][1],browser_tab_dit.loc['OTHER', 'pass_control'][1]), 
                              nobs=(browser_tab.loc['OTHER', 'pass_exposed'], browser_tab.loc['OTHER', 'pass_control']), alternative='smaller')
print(f"P-значение: {p_val:.3}") 
print('Мощьность:', get_power(browser_tab.loc['OTHER', 'pass_control'],
          browser_tab.loc['OTHER', 'pass_exposed'],
          browser_tab_dit.loc['OTHER', 'pass_control'][1],
          browser_tab_dit.loc['OTHER', 'pass_exposed'][1],
          alpha=0.05, alternative='smaller'))

P-значение: 0.158
мощьность: 0.5323486996315873


In [ ]:
## статистически значимое различие в конверсиях наблюдается только в случае "Chrome Mobile"
## в остальных случаях гипотеза об отсутствии различий в конверсиях не отвергается, однако, мощность данных экспериментов мала

## Резюме 

#### Результаты исследования:
 - Новый вариант всплывающего окна положительно сказывается на конверсии прохождения анкетирования. Статистический тест подтверждает значимость различия.
 - При текущих условиях эксперимента конверсия в контрольной группе составила 14%, а в экспериментальной группе 16%.
 - При текущих условиях эксперимента, наличие статистически значимого различия между конверсиями в контрольной и экспериментальной группе наблюдается только среди пользователей, использующих "Chrome Mobile" для просмотра сайта. Однако, в случае с пользователями других браузеров, мощность соответствующих экспериментов мала.
 
#### Рекомендации:
 - Владельцу продукта (сайта) необходимо сделать вывод о существенности изменения конверсии.
 - В случае, если изменения будут сочтены существенными, рекомендуется провести  А-А тест для проверки результатов А-В теста. 
 - Для того чтобы с большей уверенностью говорить об отсутствии статистически значимых различий между конверсиями в контрольных и экспериментальных группе среди пользователей, использующих не "Chrome Mobile" для просмотра сайта, необходимо увеличить число пользователей участвующих в эксперименте. 
 
#### Комментарий:
Все выводы сделаны, исходя из предположения, что пользователи, участвующие в эксперименте, относятся к одной когорте, выбраны случайным образом, а также не участвуют в других экспериментах, влияющих на прохождение текущего.